In [29]:
import os
from collections import OrderedDict

from pychunklbl import toolkit

def get_cli_switch_dict(cli_file_name):
    """ Usage: key_switch_dict = get_cli_switch_dict(cli_file_name)
    Get the ordered dictionary of input switch names for python file with "add_argument" - & -- names

    Args:
        cli_file_name:      python file with input args parser

    Returns:
        key_switch_dict:    {'a': argument_a, 'b': bee_arg, ...}
    """    
    key_switch_tuples_list = []
    key_switch_dict = {}

    with open(cli_file_name, 'r') as fh:
        lines = fh.readlines()

    trigger_string = 'add_argument('
    if len(lines) > 0:
        for line in lines:
            if trigger_string in line:
                start_loc = line.find(trigger_string) + len(trigger_string)
                sw_keys = line[start_loc:].strip().strip(',').split(',')
                if len(sw_keys) > 1:
                    key_switch_tuples_list.append((sw_keys[1].strip().strip('"').strip('--'), 
                                                   sw_keys[0].strip('"').strip('-')))
    if len(key_switch_tuples_list) > 0:
        key_switch_dict = OrderedDict(sorted(key_switch_tuples_list))
        
    return key_switch_dict


def cli_dot_py_switch_string_from_yml(yml_file_name, key_switch_dict):
    """ Usage: cli_switch_string = cli_dot_py_switch_string_from_yml(yml_file_name)
    open yaml file & build the command line switch string
    
    Args:
        yml_file_name:
        key_switch_dict: 
        
    Returns:
        cli_switch_string:  a string suitable for appending to python filename.py call
        
    """
    skip_list = ['run_directory', 'run_file']
    pixel_hw_names_list = ['pixel_hw', 'patch_height', 'patch_width']
    cli_switch_string = ''
    
    root_dir, yml_file_name = os.path.split(yml_file_name)
    run_pars = toolkit.get_run_parameters(root_dir, yml_file_name)
    
    for k, v in run_pars.items():
        if isinstance(v, int):
            v = '%i'%(v)
        elif isinstance(v, float):
            v = '%f'%(v)
        elif isinstance(v, bool):
            if v:
                v = True
            else:
                v = False
                
        if k in key_switch_dict:
            if k in pixel_hw_names_list:
                k2 = key_switch_dict['pixel_hw']
            else:
                k2 = key_switch_dict[k]
                
            if not k in skip_list:
                cli_switch_string += '-' + k2 + ' ' + v + ' '
            
    return cli_switch_string

# Test demo both functions

In [30]:
src_dir = '../pychunklbl'
cli_file_name = os.path.join(src_dir, 'cli.py')
key_switch_dict = get_cli_switch_dict(cli_file_name)

check_dupe_list = []
n = 0
for k, v in key_switch_dict.items():
    n += 1
    print('%3i) %25s: %s'%(n, k, v))
    if v in check_dupe_list:
        print('\n\t\t\t\tDuplicate dash switch Delicate condition')
    else:
        check_dupe_list.append(v)
    
run_parameters_directory = '../data/run_files'
run_file = 'write_mask_previews.yml'
for maybe_file in os.listdir(run_parameters_directory):
    print('\n\n', maybe_file, '\n')
    yml_file_name = os.path.join(run_parameters_directory, maybe_file)

    cli_switch_string = cli_dot_py_switch_string_from_yml(yml_file_name, key_switch_dict)
    print(cli_switch_string)

  1)              border_color: C
  2)               class_label: c
  3)             csv_file_name: L
  4)                  file_ext: l
  5)               image_level: e
  6)                    method: m
  7)          offset_data_file: D
  8)                  offset_x: x
  9)                  offset_y: y
 10)                output_dir: o
 11)       patch_select_method: P
 12)     patch_stride_fraction: s
 13)                  pixel_hw: S
 14)         rgb2lab_threshold: T
 15)        tfrecord_file_name: r
 16)                 threshold: t
 17)         thumbnail_divisor: d
 18)                   verbose: V
 19)              wsi_filename: i
 20)             wsi_floatname: f
 21)             xml_file_name: a


 write_mask_previews.yml 

-m wrte_mask_preview_set -i ../data/images/CMU-1-Small-Region.svs -o ../../run_dir -d 10 -P threshold_rgb2lab -T 80 -t 0 -e 0 -C blue -s 1.000000 -x 0 -y 0 


 registration_tfrecord_to_masked_thumb.yml 

-m tfrecord_2_masked_thumb -r ../data/tfrecords/54742

## call cli.py with run_parameters:

In [42]:
#                               get the keys to switches dictionary:
src_dir = '../pychunklbl'
cli_file_name = os.path.join(src_dir, 'cli.py')
switch_dict = get_cli_switch_dict(cli_file_name)

#                               assemble the switch srting string
pars_directory = '../data/run_files'
pars_file = os.path.join(pars_directory, 'write_mask_previews.yml')
swtch_str = cli_dot_py_switch_string_from_yml(pars_file, switch_dict)


call_string = 'python3' + ' ' + cli_file_name + ' ' + swtch_str
print(call_string)

call_string = call_string + ' ' + '&>' + ' ' + 'logmeup.txt'

python3 ../pychunklbl/cli.py -m wrte_mask_preview_set -i ../data/images/CMU-1-Small-Region.svs -o ../../run_dir -d 10 -P threshold_rgb2lab -T 80 -t 0 -e 0 -C blue -s 1.000000 -x 0 -y 0 


In [43]:
os.listdir('../../run_dir')

[]

In [47]:
os.system(call_string)

0

In [48]:
with open('logmeup.txt', 'r') as fh:
    lines = fh.readlines()
for line in lines:
    print(line.strip())

In [45]:
os.listdir('../../run_dir')

[]